In [ ]:
!pip install keras-transformer

  Created wheel for keras-transformer: filename=keras_transformer-0.40.0-py3-none-any.whl size=12305 sha256=a1d09a8ed424e04b8d242ab0223c62a41aed1cf7054438b06c874ac8dfd05adb
  Stored in directory: /root/.cache/pip/wheels/46/68/26/692ed21edd832833c3b0a0e21615bcacd99ca458b3f9ed571f
  Created wheel for keras-embed-sim: filename=keras_embed_sim-0.10.0-py3-none-any.whl size=3960 sha256=913a706d936810f3ff7c84fb7aa1f7af6abc2c2dc7afc72b2959af3964b6e61a
  Stored in directory: /root/.cache/pip/wheels/81/67/b5/d847588d075895281e1cf5590f819bd4cf076a554872268bd5
  Created wheel for keras-layer-normalization: filename=keras_layer_normalization-0.16.0-py3-none-any.whl size=4668 sha256=0d215c2485af105f032667b4e3e8a8e2b9376932eb720777a9fa4d3542391e44
  Stored in directory: /root/.cache/pip/wheels/85/5d/1c/2e619f594f69fbcf8bc20943b27d414871c409be053994813e
  Created wheel for keras-multi-head: filename=keras_multi_head-0.29.0-py3-none-any.whl size=14993 sha256=8785d66524d708a13ce7e36581c6585f0d2e0ef1940e

In [ ]:
import numpy as np
from keras_transformer import get_model, decode
from pickle import load
from google.colab import drive

In [ ]:
drive.mount('/content/drive')
filename = '/content/drive/My Drive/Goocle Collab/english-spanish.pkl'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset = load(open(filename, 'rb'))
print(dataset[120000,0])
print(dataset[120000,1])

tom is a new yorker but he doesnt have a new york accent
tom es neoyorquino pero no tiene acento de nueva york


In [ ]:
source_tokens = []
for oracion in dataset[:,0]:
  source_tokens.append(oracion.split(' '))
print(source_tokens[120000])

target_tokens = []
for oracion in dataset[:,1]:
  target_tokens.append(oracion.split(' '))
print(target_tokens[120000])

['tom', 'is', 'a', 'new', 'yorker', 'but', 'he', 'doesnt', 'have', 'a', 'new', 'york', 'accent']
['tom', 'es', 'neoyorquino', 'pero', 'no', 'tiene', 'acento', 'de', 'nueva', 'york']


In [ ]:
def build_token_dict(token_list):
  token_dict = {
      '<PAD>': 0,
      '<START>': 1,
      '<END>': 2
  }
  for tokens in token_list:
    for token in tokens:
      if token not in token_dict:
        token_dict[token] = len(token_dict)
  return token_dict

In [ ]:
source_token_dict = build_token_dict(source_tokens)
target_token_dict = build_token_dict(target_tokens)
target_token_dict_inv = {v:k for k,v in target_token_dict.items()}

In [ ]:
encoder_tokens = [['<START>'] + tokens + ['<END>'] for tokens in source_tokens]
decoder_tokens = [['<START>'] + tokens + ['<END>'] for tokens in target_tokens]
output_tokens = [tokens + ['<END>'] for tokens in target_tokens]

source_max_len = max(map(len, encoder_tokens))
target_max_len = max(map(len, decoder_tokens))

encoder_tokens = [tokens + ['<PAD>']*(source_max_len-len(tokens)) for tokens in encoder_tokens]
decoder_tokens = [tokens + ['<PAD>']*(target_max_len-len(tokens)) for tokens in decoder_tokens]
output_tokens = [tokens + ['<PAD>']*(target_max_len-len(tokens)) for tokens in output_tokens ]

In [ ]:
encoder_input = [list(map(lambda x: source_token_dict[x], tokens)) for tokens in encoder_tokens]
decoder_input = [list(map(lambda x: target_token_dict[x], tokens)) for tokens in decoder_tokens]
output_decoded = [list(map(lambda x: [target_token_dict[x]], tokens)) for tokens in output_tokens]

print(encoder_input[120000])

[1, 56, 258, 120, 197, 12666, 2914, 32, 1577, 140, 120, 197, 5385, 4287, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
modelo = get_model(
    token_num = max(len(source_token_dict),len(target_token_dict)),
    embed_dim = 32,
    encoder_num = 2,
    decoder_num = 2,
    head_num = 4,
    hidden_dim = 128,
    dropout_rate = 0.05,
    use_same_embed = False,
)
modelo.compile('adam', 'sparse_categorical_crossentropy')
modelo.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Encoder-Input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 Encoder-Token-Embedding (Embed  [(None, None, 32),  808608      ['Encoder-Input[0][0]']          
 dingRet)                        (25269, 32)]                                                     
                                                                                                  
 Encoder-Embedding (TrigPosEmbe  (None, None, 32)    0           ['Encoder-Token-Embedding[0][0]']
 dding)                                                                                           
                                                                                              

In [ ]:
x = [np.array(encoder_input), np.array(decoder_input)]
y = np.array(output_decoded)

modelo.fit(x,y, epochs=15, batch_size=32)

#filename = '/content/drive/My Drive/Goocle Collab/traductor.h5'
#model.load_weights(filename)

Epoch 1/15
3886/3886 [==============================] - 263s 64ms/step - loss: 0.6368
Epoch 2/15
3886/3886 [==============================] - 248s 64ms/step - loss: 0.3933
Epoch 3/15
3886/3886 [==============================] - 247s 64ms/step - loss: 0.3179
Epoch 4/15
3886/3886 [==============================] - 247s 64ms/step - loss: 0.2786
Epoch 5/15
3886/3886 [==============================] - 247s 64ms/step - loss: 0.2542
Epoch 6/15
3886/3886 [==============================] - 248s 64ms/step - loss: 0.2373
Epoch 7/15
3886/3886 [==============================] - 249s 64ms/step - loss: 0.2248
Epoch 8/15
3886/3886 [==============================] - 251s 65ms/step - loss: 0.2155
Epoch 9/15
3886/3886 [==============================] - 251s 65ms/step - loss: 0.2079
Epoch 10/15
3886/3886 [==============================] - 246s 63ms/step - loss: 0.2022
Epoch 11/15
3886/3886 [==============================] - 244s 63ms/step - loss: 0.1972
Epoch 12/15
3886/3886 [=============================

In [ ]:
def translate(sentence):
  sentence_tokens = [tokens + ['<END>', '<PAD>'] for tokens in [sentence.split(' ')]]
  tr_input = [list(map(lambda x: source_token_dict[x], tokens)) for tokens in sentence_tokens][0]
  decoded = decode(
      modelo, 
      tr_input, 
      start_token = target_token_dict['<START>'],
      end_token = target_token_dict['<END>'],
      pad_token = target_token_dict['<PAD>']
  )

  print('Frase original: {}'.format(sentence))
  print('Traducción: {}'.format(' '.join(map(lambda x: target_token_dict_inv[x], decoded[1:-1]))))

In [ ]:
translate('I saw a dog in the park yesterday')

Frase original: i saw a dog in the park yesterday
Traducción: ayer vi un perro en el parque


In [ ]:
translate('the function is not continuous in the origin')

Frase original: the function is not continuous in the origin
Traducción: la funcion no continuo segui en el origen


In [ ]:
translate("i do not trust in the axiom of choice")

KeyError: ignored

In [ ]:
translate('hello')

Frase original: hello
Traducción: hay nada


In [ ]:
translate('shut up')

Frase original: shut up
Traducción: callate


In [ ]:
modelo.save('traductor.h5')